In [2]:
import sys
import greenlet
import pygame
from pygame.locals import KEYDOWN, K_q, K_SPACE
import numpy as np
import sqlalchemy
import sched, time
import random
import pandas as pd
import time

pygame 2.1.2 (SDL 2.0.18, Python 3.9.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
# CONSTANTS:
SCREENSIZE = WIDTH, HEIGHT = 1200, 1800
BLACK  = (0, 0,0)
YELLOW = (244,229,0)
GREEN  = (0,142,91)
BLUE   = (42,113,176)
RED    = (227,35,34)
GREY   = (160, 60,160)
PINK   = (255,192,203)
WHITE  = (255,255,255)
GOLD   = (255,215,0)

In [4]:
#CALLING SQL DB / SAVED TRAIDS 
engine= sqlalchemy.create_engine('sqlite:///TraidsUSDTBUSD.db')
pd.read_sql('SELECT side, PnL FROM TraidsUSDTBUSD', engine)
df = pd.DataFrame(pd.read_sql('SELECT side, symbol, cummulativeQuoteQty, PnL FROM TraidsUSDTBUSD', engine))#cummulativeQuoteQty

In [5]:
#DEFINING CONDITIONS / RULES TO FILL THE ARRAY
sorted_traids = []

for i in range(len(df[1:77])):
    if  df.loc[i]['side'] == 'BUY' and df.loc[i]['symbol'] == 'BTCUSDT':
        sorted_traids.append(1)
    elif df.loc[i]['side'] == 'BUY' and df.loc[i]['symbol'] == 'BTCBUSD':
        sorted_traids.append(2)
    elif df.loc[i]['cummulativeQuoteQty'] > df.loc[i-1]['cummulativeQuoteQty']:
        sorted_traids.append(3)
    elif df.loc[i]['cummulativeQuoteQty'] < df.loc[i-1]['cummulativeQuoteQty']:
         sorted_traids.append(4)

for i in range(len(df[77:78])):
    if df['PnL'][1:99].sum() > df['PnL'][1:77].sum():
        sorted_traids.append(5)
    elif df['PnL'][1:99].sum() < df['PnL'][1:77].sum():
        sorted_traids.append(6)  
        
#DEF ARRAY
cellMAP = np.array(sorted_traids[0:77])
cellMAP = cellMAP.reshape(11,7)
cellMAP = np.transpose(cellMAP)

#DEF. GRID
_VARS = {'surf': False, 'gridWH': 1000,
         'gridOrigin': (100, 100), 'gridCells': cellMAP.shape[0], 'lineWidth': 5}

#MAINFUNKTION FOR PYGAME
def main():
    pygame.init()
    _VARS['surf'] = pygame.display.set_mode(SCREENSIZE)
    while True:
        checkEvents()
        _VARS['surf'].fill(WHITE)
        drawSquareGrid(
         _VARS['gridOrigin'], _VARS['gridWH'], _VARS['gridCells'])

#SCREENSHOT 
        for event in pygame.event.get():
             if event.type == KEYDOWN and event.key == K_SPACE:
                pygame.image.save(pygame.display.set_mode(SCREENSIZE), "screenshot"+".png")
        placeCells()
        pygame.display.update()


#ADDING CELLS :
def placeCells():
    cellBorder = 5
    celldimX = celldimY = (_VARS['gridWH']/_VARS['gridCells']) - (cellBorder*2)
    for row in range(cellMAP.shape[0]):
        for column in range(cellMAP.shape[1]):
            value = cellMAP[row][column]
            drawSquareCell(
                _VARS['gridOrigin'][0] + (celldimY*row)
                + cellBorder + (2*row*cellBorder) + _VARS['lineWidth']/2,
                _VARS['gridOrigin'][1] + (celldimX*column) 
                + cellBorder + (2*column*cellBorder) + _VARS['lineWidth']/2,
                celldimX, celldimY, value)

#DEF. COLORS FOR ITEMS IN ARRAY
def drawSquareCell(x, y, dimX, dimY, value):
   
    if value == 1: pygame.draw.rect(
                    _VARS['surf'], BLUE,
                    (x, y, dimX, dimY))
    elif value == 2: pygame.draw.rect(
                    _VARS['surf'], YELLOW,
                    (x, y, dimX, dimY))   
    elif value == 3: pygame.draw.rect(
                    _VARS['surf'], GREEN,
                    (x, y, dimX, dimY))     
    elif value == 4: pygame.draw.rect(
                    _VARS['surf'], RED,
                    (x, y, dimX, dimY))  
    elif value == 5: pygame.draw.rect(
                    _VARS['surf'], PINK,
                    (x, y, dimX, dimY))           
    else: pygame.draw.rect(
                        _VARS['surf'], GOLD,
                        (x, y, dimX, dimY)) 

def drawSquareGrid(origin, gridWH, cells):

    CONTAINER_WIDTH_HEIGHT = gridWH
    cont_x, cont_y = origin

#DRAW GRIT BRODER   
#TOP lEFT TO RIGHT
    pygame.draw.line(
      _VARS['surf'], WHITE,
      (cont_x, cont_y,),
      ((CONTAINER_WIDTH_HEIGHT)+ cont_x, cont_y,), _VARS['lineWidth'])
#BOTTOM lEFT TO RIGHT
    pygame.draw.line(
      _VARS['surf'], WHITE,
      (cont_x, CONTAINER_WIDTH_HEIGHT + cont_y +5),
      (CONTAINER_WIDTH_HEIGHT + cont_x,
       CONTAINER_WIDTH_HEIGHT + cont_y), _VARS['lineWidth'])
#LEFT TOP TO BOTTOM
    pygame.draw.line(
      _VARS['surf'], WHITE,
      (cont_x, cont_y),
      (cont_x, cont_y + CONTAINER_WIDTH_HEIGHT), _VARS['lineWidth'])
#RIGHT TOP TO BOTTOM
    pygame.draw.line(
      _VARS['surf'], WHITE,
      (CONTAINER_WIDTH_HEIGHT + cont_x, cont_y),
      (CONTAINER_WIDTH_HEIGHT + cont_x,
       CONTAINER_WIDTH_HEIGHT + cont_y), _VARS['lineWidth'])

#CELL SIZE FOR SQUARE GRID
    cellSize = CONTAINER_WIDTH_HEIGHT/cells

#VERTICAL DIVISIONS: (0,1,2) for grid(3) for example
    for x in range(cells):
        pygame.draw.line(
           _VARS['surf'], WHITE,
           (cont_x + (cellSize * x), cont_y),
           (cont_x + (cellSize * x), CONTAINER_WIDTH_HEIGHT + cont_y), 5)
#HORIZONTAl DIVISIONS
        pygame.draw.line(
          _VARS['surf'], WHITE,
          (cont_x, cont_y + (cellSize*x)),
          (cont_x + CONTAINER_WIDTH_HEIGHT, cont_y + (cellSize*x)), 5)


def checkEvents():
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            sys.exit()
        elif event.type == KEYDOWN and event.key == K_q:
            pygame.quit()
            sys.exit()


if __name__ == '__main__':
     main()


SystemExit: 

/Users/mojjo/opt/anaconda3/envs/bot1/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3405: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
